Reference:

Our code is from the tutorial created by Sigurður Skúli. The website link is: https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

In [1]:
import glob
from music21 import converter, instrument, note, chord
notes = []
for file in glob.glob("Desktop/classic_midi/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [7]:
import numpy
from keras import utils as np_utils
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
n_vocab = len(set(notes))
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [22]:
network_input.shape[2]

1

In [19]:
network_output

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
model = Sequential()
model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import h5py
from keras.callbacks import ModelCheckpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, batch_size=900, callbacks=callbacks_list)

In [10]:
model.get_weights()

[array([[-0.07206479, -0.18978478, -0.09236684, ..., -0.15561019,
         -0.0474396 , -0.1736331 ]], dtype=float32),
 array([[ 0.05882869, -0.01400619, -0.01282515, ..., -0.02105066,
         -0.01672383,  0.00502572],
        [-0.04727454, -0.00188814, -0.00465098, ..., -0.00431532,
         -0.00174077,  0.00843959],
        [-0.07286892,  0.02839655, -0.0381982 , ...,  0.00082669,
          0.03005621,  0.04435694],
        ...,
        [-0.04768886, -0.01805083,  0.00496518, ...,  0.02484941,
          0.00273518,  0.03730162],
        [ 0.05855101, -0.04440169,  0.0073748 , ..., -0.02781054,
          0.02190409,  0.01860163],
        [ 0.06692423,  0.00410607, -0.00248177, ..., -0.00644711,
         -0.01403301,  0.00903802]], dtype=float32),
 array([-0.07285231, -0.15086325, -0.08286141, ..., -0.06165717,
        -0.00421753, -0.09202705], dtype=float32),
 array([[-0.00427369, -0.03971951, -0.00157197, ...,  0.0154686 ,
         -0.06644067, -0.05550342],
        [ 0.02970402,

In [11]:
model.save_weights("model8.hdf5")

In [12]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('model8.hdf5')

In [13]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction) 
    #print(index)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern, index)
    pattern = pattern[1:len(pattern)]

In [14]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [100]:
from music21 import stream
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output2.mid')

'test_output2.mid'